In [33]:
#_____________________________________________________________________#
#---------------------- DICTIONARY MAPPER SCRIPT ---------------------#
#_____________________________________________________________________#
#---------------------------------------------------------------------#
#------------------------- CHANGES INCLUDED --------------------------#
#---------------------------------------------------------------------#
#--- * Bug Fix spelling of "ID" to "Id". -----------------------------#
#--- * Bug Fix for case insensitve matching of headers. --------------#
#--- * Bug Fix if a prefixed heading exists don't add it again. ------#
#--- * Bug Fix remove EntityList.xlsx from being processed. ----------#
#--- * New date logic, rename not copy (untested). -------------------#
#--- * New Workbench folder for combined scripts. --------------------#
#--- * Color formatted output logging. -------------------------------#
#--- * Autoresize output file columns. -------------------------------#
#---------------------------------------------------------------------#

class bcolors:
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

class CaseInsensitiveDict(dict):
    class Key(str):
        def __init__(self, key):
            str.__init__(key)
        def __hash__(self):
            return hash(self.lower())
        def __eq__(self, other):
            return self.lower() == other.lower()
    def __init__(self, data=None):
        super(CaseInsensitiveDict, self).__init__()
        if data is None:
            data = {}
        for key, val in data.items():
            self[key] = val
    def __contains__(self, key):
        key = self.Key(key)
        return super(CaseInsensitiveDict, self).__contains__(key)
    def __setitem__(self, key, value):
        key = self.Key(key)
        super(CaseInsensitiveDict, self).__setitem__(key, value)
    def __getitem__(self, key):
        key = self.Key(key)
        return super(CaseInsensitiveDict, self).__getitem__(key)

import pandas as pd
import numpy as np
from pathlib import Path
import xlsxwriter
import openpyxl
import dateutil
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

def phelp(input_string, switch):
    if switch == "B":
        return bcolors.BOLD + input_string + bcolors.ENDC
    if switch == "U":
        return bcolors.UNDERLINE + input_string + bcolors.ENDC
    if switch == "R":
        return bcolors.BOLD + bcolors.FAIL + input_string + bcolors.ENDC
    if switch == "G":
        return bcolors.BOLD + bcolors.OKGREEN + input_string + bcolors.ENDC
    if switch == "O":
        return bcolors.BOLD + bcolors.WARNING + input_string + bcolors.ENDC
    else:
        return input_string

def runner(raw_file_name):
    #Dict loading
    aff_number = str(file).split('_')[0].split('\\')[1]
    dict_frame['AffiliateNumber'].astype('str')
    filtered_dict_frame = dict_frame.loc[dict_frame['AffiliateNumber'] == aff_number]
    dictionary = CaseInsensitiveDict(pd.Series(filtered_dict_frame['MappedTo'].values,index=filtered_dict_frame['RawField']).to_dict())
    print(phelp("Loaded dictionary for affiliate {}: Found {} mappings".format(aff_number, len(dictionary)), "O"))
    
    #Variables for processing
    workbook = openpyxl.load_workbook(raw_file_name)
    worksheet = workbook.active
    number_of_columns = worksheet.max_column
    number_of_rows = worksheet.max_row - 1
    df = pd.read_excel(raw_file_name, index_col=False, dtype='object')
    out_df = pd.DataFrame(dtype=object)
    count = 0
    no_list = []
    internal_list = ["DoNotChangeAfterDate", "StopMemberNotOnFile", "StopPotentialMemberNotOnFile", "LeaveEmploymentNotOnFile", "HelpRequestTicketNumber", "AffiliateNumber", "EmployerId", "UnitId", "IsWorking"]


    # Logic Here
    for heading in df.columns:
        if heading in dictionary:
            df.rename(columns={heading: dictionary[heading]}, inplace=True)
            count = count + 1
        else:
            no_list.append(heading)
        if "Date" in heading:
                out_df[heading] = pd.to_datetime(out_df[heading])
                out_df[heading] = out_df[heading].dt.strftime('%m/%d/%Y')

    for header in internal_list:
        if not header in df.columns:
            out_df[header] = ""
    frames = [out_df, df]
    df = pd.concat(frames)

    #output file variables
    filename = (str(raw_file_name).split('.xlsx')[0] + '_DictionaryMapped.xlsx')
    writer = pd.ExcelWriter(filename)
    
    print(phelp("Matched and updated {}/{} headers".format(count, number_of_columns), "O"))
    print(phelp("No matches found for {} headers: ".format(len(no_list)), "R") + phelp("{}".format(no_list), "B"))
    
    #Write dataframe to excel minus headers
    df.to_excel(writer, engine='xlsxwriter', index=False, startrow=1, header=False)


    #For loop magic to re-add the headers in the right format
    out_workbook  = writer.book
    out_worksheet = writer.sheets['Sheet1']
    column_list = df.columns

    currency_format = out_workbook.add_format({'num_format': '$#,##0.00'})
    date_format = out_workbook.add_format({'num_format': 'm/d/yy'})
    text_format = out_workbook.add_format({'num_format': '@'})

    for idx, val in enumerate(column_list):
        out_worksheet.write(0, idx, val)

    for idx, val in enumerate(column_list):
        if "CopeAmount" in str(val):
            #print(idx, val)
            out_worksheet.set_column(idx, idx, 10, currency_format)
        if "Date" in str(val):
            out_worksheet.set_column(idx, idx, 10, date_format)
            #print(idx, val)
        else:
            #print(idx, val)
            out_worksheet.set_column(idx, idx, 10, text_format)


    # Auto-adjust columns' width
    for column in df:
        column_width = max(df[column].astype(str).map(len).max(), len(column))
        col_idx = df.columns.get_loc(column)
        writer.sheets['Sheet1'].set_column(col_idx, col_idx, column_width)
    
    #Saves the output file to IO
    writer.save()
    print("Done! Created output file: " + phelp("{}".format(filename.split('\\')[-1]), "G"))
    

#Open file for dictionary mappings
dict_file = Path('./Templates/FileFieldDictionary.xlsx')
dict_file_name = dict_file.resolve()
dict_frame = pd.read_excel(dict_file_name, index_col=False, dtype='object')

#Open raw file for data access
clean_pathlist = []
pathlist = list((Path('./').glob('**/Workbench/*.xlsx')))
for path in pathlist:
    if ((not "KnackBuild".casefold() in path.stem.casefold()) 
        and (not "DictionaryMapped".casefold() in path.stem.casefold())
        and (not "EntityList".casefold() in path.stem.casefold())):
        if not "~$" in path.stem:
            clean_pathlist.append(path)

total = len(clean_pathlist)
current = 1
print("------------------------------------")
for file in clean_pathlist:
    raw_file_name = file.resolve()
    print("Starting file {}/{}: ".format(current, total) + phelp("{}".format(str(file).split('\\')[1]), "G"))
    runner(raw_file_name)
    print("------------------------------------")
    current = current + 1


------------------------------------
Starting file 1/1: 0003_23718_20211115.xlsx
Loaded dictionary for affiliate 0003: Found 0 mappings
Matched and updated 0/5 headers
No matches found for 5 headers: ['LastName', 'FirstName', 'Work Location Name', 'WorkEmail', 'PhoneNumber']
Done! Created output file: 0003_23718_20211115_DictionaryMapped.xlsx
------------------------------------


In [36]:
#_____________________________________________________________________#
#------------------ NOTKNACK STAGING TEMPLATE SCRIPT -----------------#
#_____________________________________________________________________#
#---------------------------------------------------------------------#
#------------------------- CHANGES INCLUDED --------------------------#
#---------------------------------------------------------------------#
#--- * Bug Fix infinite file loop from dictionary mapper. ------------#
#--- * Bug Fix for case insensitve matching of headers. --------------#
#--- * Bug Fix close output file properly (couldn't delete prev). ----#
#--- * Bug Fix remove EntityList from being processed. ---------------#
#--- * New Workbench folder for combined scripts. --------------------#
#--- * Color formatted output logging. -------------------------------#
#--- * Autoresize output file columns. -------------------------------#
#---------------------------------------------------------------------#

class bcolors:
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


import pandas as pd
import numpy as np
from pathlib import Path
import xlsxwriter
import openpyxl
import dateutil
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

def phelp(input_string, switch):
    if switch == "B":
        return bcolors.BOLD + input_string + bcolors.ENDC
    if switch == "U":
        return bcolors.UNDERLINE + input_string + bcolors.ENDC
    if switch == "R":
        return bcolors.BOLD + bcolors.FAIL + input_string + bcolors.ENDC
    if switch == "G":
        return bcolors.BOLD + bcolors.OKGREEN + input_string + bcolors.ENDC
    if switch == "O":
        return bcolors.BOLD + bcolors.WARNING + input_string + bcolors.ENDC
    else:
        return input_string
    

def get_maximum_rows(*, sheet_object):
    rows = 0
    for max_row, row in enumerate(sheet_object, 1):
        if not all(col.value is None for col in row):
            rows += 1
    return rows

def runner(worksheet):
    #Variables for processing
    max_rows = get_maximum_rows(sheet_object=worksheet)
    number_of_columns = worksheet.max_column
    number_of_rows = max_rows - 1

    
    df = pd.read_excel(raw_file_name, index_col=False)
    df_lower = df.copy(deep=False)
    df_lower.columns = df_lower.columns.str.lower()
    out_df = pd.DataFrame(dtype=object)
    count = 0

    # Logic Here for adding every heading,
    # if the heading exists in the raw file the rows are copied
    # if the heading doesn't exist it is applied with blank rows
    for heading in template_list:
        if heading.lower() in df_lower:
            out_df[heading] = df_lower[heading.lower()].values.tolist()
            df_lower.drop(heading.lower(), axis=1, inplace=True)
            count = count + 1
        else:
            out_df[heading] = ""
        if "Date" in heading:
                out_df[heading] = pd.to_datetime(out_df[heading])
                out_df[heading] = out_df[heading].dt.strftime('%m/%d/%Y')


    diff_list = df_lower.columns
    for col in diff_list:
        print(phelp("NO MATCH, APPENDING COLUMN: ", "R") + phelp("{}".format(col), "B"))
        out_df[col] = df_lower[col].values.tolist()
        count = count + 1
    
    #output file variables
    filename = (str(raw_file_name).split('.xlsx')[0].split('_DictionaryMapped')[0] + '_KnackBuild.xlsx')
    writer = pd.ExcelWriter(filename)

    #Write dataframe to excel minus headers
    out_df.to_excel(writer, engine='xlsxwriter', index=False, startrow=1, header=False)


    #For loop magic to re-add the headers in the right format
    out_workbook  = writer.book
    out_worksheet = writer.sheets['Sheet1']
    column_list = out_df.columns

    currency_format = out_workbook.add_format({'num_format': '$#,##0.00'})
    date_format = out_workbook.add_format({'num_format': 'm/d/yy'})
    text_format = out_workbook.add_format({'num_format': '@'})

    for idx, val in enumerate(column_list):
        out_worksheet.write(0, idx, val)

    for idx, val in enumerate(column_list):
        if "CopeAmount" in str(val):
            #print(idx, val)
            out_worksheet.set_column(idx, idx, 10, currency_format)
        if "Date" in str(val):
            out_worksheet.set_column(idx, idx, 10, date_format)
            #print(idx, val)
        else:
            #print(idx, val)
            out_worksheet.set_column(idx, idx, 10, text_format)

    # Auto-adjust columns' width
    for column in out_df:
        column_width = max(out_df[column].astype(str).map(len).max(), len(column))
        col_idx = out_df.columns.get_loc(column)
        writer.sheets['Sheet1'].set_column(col_idx, col_idx, column_width)

    #Saves the output file to IO
    writer.save()

    #Validation print logs and variables
    workbook = openpyxl.load_workbook(filename)
    worksheet = workbook["Sheet1"]
    final_number_of_rows = worksheet.max_row - 1

    print(phelp("Columns in raw file: {}. Total data columns added to output file: {}".format(number_of_columns, count), "O"))
    print(phelp("Data rows in raw file: {}. Total data rows added to output file: {}".format(number_of_rows, final_number_of_rows), "O"))
    print("Done! Created output file: " + phelp("{}".format(filename.split('\\')[-1]), "G"))
    
    workbook.close()

#Read template file and make list of column headers
template_path = Path('./Templates/NotKnackTemplate.txt')
template_file = open(template_path.resolve(), 'r')
template = template_file.read()
template_list = template.split(",")

#Open raw file for data access
clean_pathlist = []
pathlist = list((Path('./').glob('**/WorkBench/*.xlsx')))
for path in pathlist:
    if "DictionaryMapped".casefold() in path.stem.casefold():
        if not "~$" in path.stem:
            clean_pathlist.append(path)

total = len(clean_pathlist)
current = 1
print("------------------------------------")
for file in clean_pathlist:
    raw_file_name = file.resolve()
    print("Starting file {}/{}: ".format(current, total) + phelp("{}".format(str(file).split('\\')[1]), "G"))
    workbook = openpyxl.load_workbook(raw_file_name)
    worksheet = workbook.active
    runner(worksheet)
    print("------------------------------------")
    current = current + 1
    

------------------------------------
Starting file 1/1: 0003_23718_20211115_DictionaryMapped.xlsx
NO MATCH, APPENDING COLUMN: work location name
Columns in raw file: 14. Total data columns added to output file: 14
Data rows in raw file: 126. Total data rows added to output file: 126
Done! Created output file: 0003_23718_20211115_KnackBuild.xlsx
------------------------------------
